# Video Summarizer 모델 학습/평가 코드

In [16]:
import os
import math
import numpy as np
import pandas as pd

## 경로 설정 & 파라미터 설정

In [17]:
# dataset_dir = r'E:\Work\YasuoNet\data\dataset14_sl3_vsr2_vw64_vh64_asr22050_mfcc'
dataset_dir = './data'
ckpt_dir = 'ckpt'

x_includes = ['video', 'audio']
batch_size = 139
# for basic model
#x_expand = 0
# for sequence model
x_expand = 2    # 앞 2개, 뒤 2개 segment 포함

## 데이터 로더 생성

In [18]:
from module.data_loader import DataLoader

data_loader = DataLoader(dataset_dir, x_includes=x_includes, x_expand=x_expand)

data_config = data_loader.get_metadata()['config']
input_shape_dict = data_loader.get_metadata()['data_shape']

## 모델의 모든 정보를 온전하게 저장 / 복원
모델의 가중치 뿐만아니라 모든 레이어 구성 정보를 저장하여 추후 모델 선언부가 없어도 불러와서 사용 가능

### 모델 복원

In [19]:
checkpoint_name = 'ckpt-20200826-223214-0007-0.7341_model'
model_path = os.path.join(ckpt_dir, checkpoint_name + '.h5')
print(model_path)

ckpt\ckpt-20200826-223214-0007-0.7341_model.h5


In [20]:
from tensorflow.keras.models import load_model

model_restored = load_model(model_path)

In [21]:
model_restored.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 6, 64, 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 40, 13 0                                            
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, None, 49152)  1312        input_1[0][0]                    
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 10400)  80          input_2[0][0]                    
_______________________________________________________________________________________

### 복원된 모델 테스트

In [22]:
from module.trainer import Trainer

trainer = Trainer(model_restored, data_loader, ckpt_dir)
y_pred = trainer.test_prediction(batch_size)

print(f'pred_1: {y_pred.sum()}')
print()


pred_1: 693



## 예측값 파일에 기록하기

In [25]:
import shutil
import glob
from moviepy.editor import VideoFileClip, concatenate_videoclips

def save_predictions():
    from data_converter import to_hms
    
    segment_length = data_loader.get_metadata()['config']['segment_length']
    
    segment_df = data_loader.test_segment_df.copy()
    segment_df['pred'] = y_pred
    segment_df['start_sec'] = (segment_df['index'] * segment_length)
    segment_df['end_sec'] = ((segment_df['index'] + 1) * segment_length)
    
    mp4_dir = '.' #해당 mp4 파일 폴더
    start = np.array(segment_df['start_sec'][segment_df['pred'] == 1])
    end = np.array(segment_df['end_sec'][segment_df['pred'] == 1])
    name = "raw"
    
    i=1
    while(i<len(end)):
        if end[i] - 3 == end[i-1]:
            end[i-1] = end[i]
            start = np.delete(start, i)
            end = np.delete(end, i)
        else:
            i+=1

    clip = VideoFileClip(os.path.join(mp4_dir,name+".mp4"))
    subclips = []
    for i in range(len(start)):
        start_lim = start[i]
        end_lim = end[i]
        subclips.append(clip.subclip(start_lim, end_lim))
    final_clip=concatenate_videoclips(subclips)
    final_clip.write_videofile("./Highlights" + name + ".mp4") #Enter the desired output highlights filename.
    for i in subclips:
        i.close()
    clip.close()

In [26]:
save_predictions()